In [1]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'nvidia-docker2' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-doc' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-headless-390' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-gcc' for glob 'nvidia*'
Note, selecting 'nvidia-headless-418' for glob 'nvidia*'
Note, selecting 'nvidia-headless-430' for glob 'nvidia*'
Note, selecting 'nvidia-headless-435' for glob 'nvidia*'
Note, selecting 'nvidia-headless-440' for glob 'nvidia*'
Note, selecting 'nvidia-headless-450' for glob 'nvidia*'
Note, selecting 'nvidia-headless-455' for glob 'nvidia*'
Note, selecting 'nvidia-headless-460' for glob 'nvidia*'
Note, selecting 'nvidia-headless-465' for glob 'nvidia*'
Note, selecting 'nvidia-headless-470' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-470-server' for glob 'nvidia*'
Note, selecting 'nvidia-headless-495' f

In [2]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/debian11/x86_64/cuda-keyring_1.1-1_all.deb
!sudo dpkg -i cuda-keyring_1.1-1_all.deb
!sudo add-apt-repository contrib
!apt-get update
!sudo apt-get -y install cuda

--2023-08-20 05:59:15--  https://developer.download.nvidia.com/compute/cuda/repos/debian11/x86_64/cuda-keyring_1.1-1_all.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4148 (4.1K) [application/x-deb]
Saving to: ‘cuda-keyring_1.1-1_all.deb’

cuda-keyring_1.1-1_ 100%[===================>]   4.05K  --.-KB/s    in 0s      

2023-08-20 05:59:15 (286 MB/s) - ‘cuda-keyring_1.1-1_all.deb’ saved [4148/4148]

Selecting previously unselected package cuda-keyring.
(Reading database ... 119160 files and directories currently installed.)
Preparing to unpack cuda-keyring_1.1-1_all.deb ...
Unpacking cuda-keyring (1.1-1) ...
Setting up cuda-keyring (1.1-1) ...
Unable to handle repository shortcut 'contrib'
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cloud.r-project.org/

In [3]:
!nvidia-smi

Sun Aug 20 05:59:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import torch
torch.cuda.is_available()


True

In [5]:
!nvcc --version

/bin/bash: line 1: nvcc: command not found


In [6]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [7]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.0 MB/s eta 0:00:00


In [8]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv



In [9]:
lyrics = pd.read_csv('/content/drive/MyDrive/lyricspark/lyrics-data.csv')
lyrics

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt
...,...,...,...,...,...
379926,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en
379927,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en
379928,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en
379929,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en


In [10]:
lyrics = lyrics[lyrics['language']=='en']

In [11]:

lyrics

,ALink,SName,SLink,Lyric,language
69,/ivete-sangalo/,Careless Whisper,/ivete-sangalo/careless-whisper.html,I feel so unsure\nAs I take your hand and lead...,en
86,/ivete-sangalo/,Could You Be Loved / Citação Musical do Rap: S...,/ivete-sangalo/could-you-be-loved-citacao-musi...,"Don't let them fool, ya\nOr even try to school...",en
88,/ivete-sangalo/,Cruisin' (Part. Saulo),/ivete-sangalo/cruisin-part-saulo.html,"Baby, let's cruise, away from here\nDon't be c...",en
111,/ivete-sangalo/,Easy,/ivete-sangalo/easy.html,"Know it sounds funny\nBut, I just can't stand ...",en
140,/ivete-sangalo/,For Your Babies (The Voice cover),/ivete-sangalo/for-your-babies-the-voice-cover...,You've got that look again\nThe one I hoped I ...,en
...,...,...,...,...,...
379926,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en
379927,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en
379928,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en
379929,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en


In [12]:
#Only keep popular artists, with genre Rock/Pop and popularity high enough
artists = pd.read_csv('/content/drive/MyDrive/lyricspark/artists-data.csv')
artists


,Artist,Genres,Songs,Popularity,Link
0,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
1,Chiclete com Banana,Axé,268.0,3.8,/chiclete-com-banana/
2,Banda Eva,Axé; Romântico; Reggae,215.0,2.3,/banda-eva/
3,É O Tchan,Axé,129.0,1.6,/e-o-tchan/
4,Claudia Leitte,Pop; Axé; Romântico,167.0,1.5,/claudia-leitte/
...,...,...,...,...,...
4163,Miriam Makeba,World Music; Black Music; Blues,17.0,0.0,/miriam-makeba/
4164,Freddie Aguilar,World Music,61.0,0.0,/freddie-aguilar/
4165,Amadou & Mariam,World Music,14.0,0.0,/amadou-mariam/
4166,Magic System,World Music; Gospel/Religioso,16.0,0.0,/magic-system/


In [13]:
artists = artists[(artists['Genres'].isin(['Rock'])) & (artists['Popularity']>5)]

In [14]:
artists

,Artist,Genres,Songs,Popularity,Link
3446,4 Non Blondes,Rock,15.0,10.1,/4-non-blondes/
3448,David Bowie,Rock,463.0,6.8,/david-bowie/
3449,Bruce Springsteen,Rock,505.0,5.2,/bruce-springsteen/


In [15]:
df = lyrics.merge(artists[['Artist', 'Genres', 'Link']], left_on='ALink', right_on='Link', how='inner')

In [16]:
df

,ALink,SName,SLink,Lyric,language,Artist,Genres,Link
0,/4-non-blondes/,What's Up,/4-non-blondes/whats-up.html,Twenty-five years and my life is still\nTrying...,en,4 Non Blondes,Rock,/4-non-blondes/
1,/4-non-blondes/,Spaceman,/4-non-blondes/spaceman.html,Starry night bring me down\nTill I realize the...,en,4 Non Blondes,Rock,/4-non-blondes/
2,/4-non-blondes/,Pleasantly Blue,/4-non-blondes/pleasantly-blue.html,Every time you wake in the mornin'\nAnd you st...,en,4 Non Blondes,Rock,/4-non-blondes/
3,/4-non-blondes/,I'm The One,/4-non-blondes/im-the-one.html,Ah-hah!\nWoo!\nAh-ha-ha-ha-ha-ha!\nWe came her...,en,4 Non Blondes,Rock,/4-non-blondes/
4,/4-non-blondes/,Dear Mr. President,/4-non-blondes/dear-mr-president.html,I'm looking outside of my windows\nThe view th...,en,4 Non Blondes,Rock,/4-non-blondes/
...,...,...,...,...,...,...,...,...
949,/bruce-springsteen/,You've Got It,/bruce-springsteen/youve-got-it.html,"No one ever found it, ain't no school ever tau...",en,Bruce Springsteen,Rock,/bruce-springsteen/
950,/bruce-springsteen/,Youngstown,/bruce-springsteen/youngstown.html,Here in northeast Ohio\nBack in eighteen-o-thr...,en,Bruce Springsteen,Rock,/bruce-springsteen/
951,/bruce-springsteen/,Your Love,/bruce-springsteen/your-love.html,She promises to be yours forever and always\nA...,en,Bruce Springsteen,Rock,/bruce-springsteen/
952,/bruce-springsteen/,Your Own Worst Enemy,/bruce-springsteen/your-own-worst-enemy.html,Your Own Worst Enemy\n\n\nYou can't sleep at n...,en,Bruce Springsteen,Rock,/bruce-springsteen/


In [17]:
df = df.drop(columns=['ALink','SLink','language','Link'])

In [18]:
#Drop the songs with lyrics too long (after more than 1024 tokens, does not work)
df = df[df['Lyric'].apply(lambda x: len(x.split(' ')) < 350)]

In [19]:

len(df)

905

In [47]:
#Create a very small test set to compare generated text with the reality
test_set = df.sample(n = 50)
df = df.loc[~df.index.isin(test_set.index)]

#Reset the indexes
test_set = test_set.reset_index()
df = df.reset_index()

In [21]:
test_set['True_end_lyrics'] = test_set['Lyric'].str.split().str[-20:].apply(' '.join)
test_set['Lyric'] = test_set['Lyric'].str.split().str[:-20].apply(' '.join)

In [22]:

test_set.head()

,index,SName,Lyric,Artist,Genres,True_end_lyrics
0,169,Here Comes The Night,Ohhhhh... here it comes Here comes the night Y...,David Bowie,Rock,"comes the Here comes the night yaeh Lonely, lo..."
1,393,There Is A Happy Land,There is a happy land where only children live...,David Bowie,Rock,"your chance and now the doors are closed sir, ..."
2,882,This Hard Land,Hey there mister can you tell me what happened...,Bruce Springsteen,Rock,"it Stay hard, stay hungry, stay alive If you c..."
3,611,Gypsy Biker,Gypsy Biker The speculators made their money o...,Bruce Springsteen,Rock,la la la La la la la La la la la La la la la L...
4,679,Let's Go,I hear the guitars ringing out Ringing out dow...,Bruce Springsteen,Rock,new life I wanna start a new life I wanna ... ...


In [23]:
class SongLyrics(Dataset):

    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in df['Lyric']:
          self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))

        if truncate:
            self.lyrics = self.lyrics[:20000]
        self.lyrics_count = len(self.lyrics)

    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]


In [24]:
dataset = SongLyrics(df['Lyric'], truncate=True, gpt2_type="gpt2")

In [25]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [26]:
#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [27]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=20, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):

    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [28]:
#Train the model on the specific data we have
model = train(dataset, model, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training epoch 0
0


895it [01:16, 11.64it/s]


Training epoch 1
tensor(3.3796, device='cuda:0', grad_fn=<NllLossBackward0>)


895it [01:15, 11.81it/s]


...

895it [01:15, 11.88it/s]


Training epoch 18
tensor(1.2522, device='cuda:0', grad_fn=<NllLossBackward0>)


895it [01:15, 11.84it/s]


Training epoch 19
tensor(1.3012, device='cuda:0', grad_fn=<NllLossBackward0>)


895it [01:15, 11.85it/s]


In [29]:
 #Save the model to a pkl or something so it can be reused later on
torch.save(model, '/content/drive/MyDrive/lyricspark/model.pt')


In [30]:
#Load the model to use it
model = torch.load('/content/drive/MyDrive/lyricspark/model.pt')

In [31]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):

    model.eval()

    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False

            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break

            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>"
              generated_list.append(output_text)

    return generated_list


In [35]:
#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_lyrics = []
  for i in range(len(test_data)):
    x = generate(model.to('cpu'), tokenizer, test_data['Lyric'][i], entry_count=1)
    generated_lyrics.append(x)
  return generated_lyrics

In [48]:
generated_lyrics = text_generation(test_set)

100%|██████████| 1/1 [00:24<00:00, 24.16s/it]


In [49]:
#Loop to keep only generated text and add it as a new column in the dataframe
my_generations=[]

for i in range(len(generated_lyrics)):
  a = test_set['Lyric'][i].split()[-30:] #Get the matching string we want (30 words)
  b = ' '.join(a)
  c = ' '.join(generated_lyrics[i]) #Get all that comes after the matching string
  my_generations.append(c.split(b)[-1])

test_set['Generated_lyrics'] = my_generations


In [50]:
#Finish the sentences when there is a point, remove after that
final=[]

for i in range(len(test_set)):
  to_remove = test_set['Generated_lyrics'][i].split('.')[-1]
  final.append(test_set['Generated_lyrics'][i].replace(to_remove,''))

test_set['Generated_lyrics'] = final
test_set.head()

,level_0,index,SName,Lyric,Artist,Genres,Generated_lyrics
0,788,838,Surprise Surprise,"Well, surprise, surprise, surprise\nYea, surpr...",Bruce Springsteen,Rock,
1,485,509,Be True,Your scrapbook's filled with pictures of all y...,Bruce Springsteen,Rock,Your scrapbook's filled with pictures of all y...
2,831,884,This Land Is Your Land,Well I rode that ribbon highway\nI saw above m...,Bruce Springsteen,Rock,
3,881,939,Wreck On The Highway,Last night I was out driving\nComing home at t...,Bruce Springsteen,Rock,
4,78,80,Because You're Young,Psychodelicate girl - come out to play\nLittle...,David Bowie,Rock,Psychodelicate girl - come out to play\nLittle...


In [51]:
test_set['Generated_lyrics'][5]

''

In [52]:
test_set['True_end_lyrics'][5]

KeyError: ignored